In [ ]:
from scipy.io import arff
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load the dataset
data = arff.loadarff('column_diagnosis.arff')
df = pd.DataFrame(data[0])

# 1

# 2

# 3

# 4